# 量化调试

In [1]:
# 设置警告
import warnings
warnings.filterwarnings(
    action='ignore',
    category=DeprecationWarning,
    module=r'.*'
)
warnings.filterwarnings(
    action='default',
    module=r'torch.ao.quantization'
)

In [2]:
from copy import deepcopy
from torchvision.models.resnet import resnet18, ResNet18_Weights
from torchvision.models.quantization import resnet18 as qresnet18, ResNet18_QuantizedWeights
from tools.utils import size_of_model, accuracy, evaluate
from tools.imagenet import Dataset

In [3]:
from dataclasses import dataclass
from pathlib import Path

In [ ]:
from d2py.quantum.graph.tree import Bunch

cfg = Bunch({k: Bunch(v) for k, v in Project(args.model).cfg.items()})

In [ ]:
saved_model_dir = 'data/' # 模型存储路径
train_batch_size = 30 # 训练样本批量大小
eval_batch_size = 50 # 测试样本批量大小
dataset = Dataset()
data_loader = dataset.train_loader(train_batch_size)
data_loader_test = dataset.test_loader(eval_batch_size)
example_inputs = (next(iter(data_loader))[0])
# PTQ
eager_quantized_model = qresnet18(weights=ResNet18_QuantizedWeights.DEFAULT, quantize=True).eval()

In [ ]:
# criterion = nn.CrossEntropyLoss()
float_model = resnet18(weights=ResNet18_Weights.DEFAULT)
float_model = float_model.to("cpu").eval()
# 深度复制模型，因为需要保持原始模型
model_to_quantize = deepcopy(float_model)

In [ ]:
from torch.ao.quantization import get_default_qconfig, QConfigMapping
# 旧的 'fbgemm' 仍然可用，但 'x86' 是推荐的默认值。
qconfig = get_default_qconfig("x86")
qconfig_mapping = QConfigMapping().set_global(qconfig)

## PTQ

In [ ]:
from torch.ao.quantization.quantize_fx import prepare_fx #, convert_fx, fuse_fx

prepared_model = prepare_fx(model_to_quantize, qconfig_mapping, example_inputs)

### 校准

In [ ]:
import torch

def calibrate(model, data_loader, num=200):
    model.eval()
    with torch.no_grad():
        for k, (image, _) in enumerate(data_loader):
            if k > num:
                break
            model(image)
calibrate(prepared_model, data_loader)  # 在样本数据上运行校准

### 变换 PTQ 模型

In [ ]:
from torch.ao.quantization.quantize_fx import convert_fx

quantized_model = convert_fx(prepared_model)

# PTQ 与 QAT 实践

本文主要介绍如何使用 PyTorch 将浮点模型转换为 PTQ 或者 QAT 模型。

## 背景

{guilabel}`目标`：快速将浮点模型转换为 PTQ 或者 QAT 模型。

### 读者

本教程适用于会使用 PyTorch 编写 CNN 等模块的的算法工程师。

### 环境配置

本文使用 Python 3.10.0 （其他版本请自测），暂时仅 Linux 平台被测试。

### 评估 PTQ 模型

# PTQ 与 QAT 实践

本文主要介绍如何使用 PyTorch 将浮点模型转换为 PTQ 或者 QAT 模型。

## 背景

{guilabel}`目标`：快速将浮点模型转换为 PTQ 或者 QAT 模型。

### 读者

本教程适用于会使用 PyTorch 编写 CNN 等模块的的算法工程师。

### 环境配置

本文使用 Python 3.10.0 （其他版本请自测），暂时仅 Linux 平台被测试。

# PTQ 与 QAT 实践

本文主要介绍如何使用 PyTorch 将浮点模型转换为 PTQ 或者 QAT 模型。

## 背景

{guilabel}`目标`：快速将浮点模型转换为 PTQ 或者 QAT 模型。

### 读者

本教程适用于会使用 PyTorch 编写 CNN 等模块的的算法工程师。

### 环境配置

本文使用 Python 3.10.0 （其他版本请自测），暂时仅 Linux 平台被测试。

# PTQ 与 QAT 实践

本文主要介绍如何使用 PyTorch 将浮点模型转换为 PTQ 或者 QAT 模型。

## 背景

{guilabel}`目标`：快速将浮点模型转换为 PTQ 或者 QAT 模型。

### 读者

本教程适用于会使用 PyTorch 编写 CNN 等模块的的算法工程师。

### 环境配置

本文使用 Python 3.10.0 （其他版本请自测），暂时仅 Linux 平台被测试。

In [ ]:
def size_of_model(model):
    """返回模型大小"""
    import os
    if isinstance(model, torch.jit.RecursiveScriptModule):
        torch.jit.save(model, "temp.p")
    else:
        torch.jit.save(torch.jit.script(model), "temp.p")
    size = os.path.getsize('temp.p')
    os.remove("temp.p")
    return size

In [ ]:
print(f"量化前模型大小: {size_of_model(float_model)/(1<<20)} MB")
print(f"量化后模型大小: {size_of_model(quantized_model)/(1<<20)} MB")

In [ ]:
top1, top5 = evaluate(quantized_model, data_loader_test)
print(f"[序列化前] 测试数据集的准确性评估: {top1.avg: 2.5g}, {top5.avg: 2.5g}")

In [ ]:
fx_graph_mode_model_file_path = saved_model_dir + "resnet18_fx_graph_mode_quantized.pth"
torch.jit.save(torch.jit.script(quantized_model), fx_graph_mode_model_file_path)

In [ ]:
fx_graph_mode_model_file_path = saved_model_dir + "resnet18_fx_graph_mode_quantized.pth"
torch.jit.save(torch.jit.script(quantized_model), fx_graph_mode_model_file_path)
loaded_quantized_model = torch.jit.load(fx_graph_mode_model_file_path)
top1, top5 = evaluate(loaded_quantized_model, data_loader_test)
print(f"[序列化后] 测试数据集的准确性评估: {top1.avg: 2.5g}, {top5.avg: 2.5g}")

In [ ]:
print(f"baseline 模型大小: {size_of_model(float_model)/(1<<20)} MB")